In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# 1. Load dataset
df = pd.read_csv("/kaggle/input/augmented-epileptic-seizure-dataset/EEG_seizure_data_augmented.csv")
# df.head()

# 2. Drop index column if present
if 'Unnamed' in df.columns:
    df.drop('Unnamed', axis=1, inplace=True)

# 3. Convert multiclass to binary:
# Class 1 = seizure (label 1), Classes 2–5 = non-seizure (label 0)
df['y_binary'] = (df['y'] == 1).astype(int)

# 4. Define feature columns and extract data
feature_cols = [f'X{i}' for i in range(1, 179)]  # X1 to X178
X = df[feature_cols].values                     # Shape: (11500, 178)
y = df['y_binary'].values                       # Shape: (11500,)

# 5. Standardize features (mean 0, std 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 6. Reshape for RNN input: (samples, timesteps, features_per_timestep)
# Here: each sample has 178 timesteps, 1 feature per timestep
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)  # (11500, 178, 1)

# 7. Split data: 60% train, 20% validation, 20% test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_reshaped, y, test_size=0.2, stratify=y, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

# 8. Compute class weights for imbalanced data
class_weights = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# 9. Print shapes and weights
print("Data shapes:")
print(f"  Train:      {X_train.shape}")
print(f"  Validation: {X_val.shape}")
print(f"  Test:       {X_test.shape}")
print(f"Class weights: {class_weight_dict}")

Data shapes:
  Train:      (13800, 178, 1)
  Validation: (4600, 178, 1)
  Test:       (4600, 178, 1)
Class weights: {0: 0.625, 1: 2.5}


# Model Defination


In [2]:
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, BatchNormalization, Bidirectional, Multiply, Permute, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

def build_model(hp):
    input_layer = Input(shape=(178, 1))

    x = Bidirectional(
        GRU(hp.Int('gru_units1', 32, 128, step=16), return_sequences=True, activation='tanh')
    )(input_layer)
    x = Dropout(hp.Float('dropout1', 0.2, 0.5, step=0.1))(x)
    x = BatchNormalization()(x)

    x = Bidirectional(
        GRU(hp.Int('gru_units2', 16, 64, step=16), return_sequences=True, activation='tanh')
    )(x)
    x = Dropout(hp.Float('dropout2', 0.2, 0.5, step=0.1))(x)
    x = BatchNormalization()(x)

    # Temporal Attention Mechanism
    attention_weights = Dense(1, activation='tanh')(x)                         # shape: (batch, time_steps, 1)
    attention_weights = Lambda(lambda z: K.softmax(z, axis=1))(attention_weights)  # softmax over time
    context_vector = Multiply()([x, attention_weights])                        # weighted sequence
    context_vector = Lambda(lambda z: K.sum(z, axis=1))(context_vector)        # sum over time steps

    x = Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu')(context_vector)
    x = Dropout(hp.Float('dropout3', 0.2, 0.5, step=0.1))(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-3, 1e-4, 5e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


2025-05-26 09:06:27.334136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748250387.532785      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748250387.595768      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# HT

In [3]:
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score


# 2. Setup tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='seizure_tuning',
    project_name='rnn_seizure_detection'
)

# 3. Early stopping
stop_early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 4. Hyperparameter search
tuner.search(X_train, y_train,
             epochs=30,
             validation_data=(X_val, y_val),
             callbacks=[stop_early],
             batch_size=32)

# 5. Get best model and best hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:", best_hp.values)

# 6. Retrain on train+val data (optional)
best_model.fit(X_train, y_train,
               epochs=30,
               validation_data=(X_val, y_val),
               batch_size=32,
               callbacks=[stop_early])

# 7. Accuracy evaluation
train_preds = (best_model.predict(X_train) > 0.5).astype(int)
val_preds = (best_model.predict(X_val) > 0.5).astype(int)
test_preds = (best_model.predict(X_test) > 0.5).astype(int)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
test_acc = accuracy_score(y_test, test_preds)

print(f"Training Accuracy:   {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy:       {test_acc:.4f}")


Trial 20 Complete [00h 05m 44s]
val_accuracy: 0.997826099395752

Best val_accuracy So Far: 0.997826099395752
Total elapsed time: 01h 57m 00s
Best hyperparameters: {'gru_units1': 32, 'dropout1': 0.4, 'gru_units2': 64, 'dropout2': 0.30000000000000004, 'dense_units': 96, 'dropout3': 0.4, 'lr': 0.0005}
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


432/432 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.9961 - loss: 0.0089 - val_accuracy: 0.9952 - val_loss: 0.0125
Epoch 2/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.9951 - loss: 0.0135 - val_accuracy: 0.9959 - val_loss: 0.0124
Epoch 3/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9973 - loss: 0.0087 - val_accuracy: 0.9972 - val_loss: 0.0097
Epoch 4/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.9964 - loss: 0.0126 - val_accuracy: 0.9963 - val_loss: 0.0093
Epoch 5/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.9973 - loss: 0.0094 - val_accuracy: 0.9980 - val_loss: 0.0063
Epoch 6/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9973 - loss: 0.0083 - val_accuracy: 0.9976 - val_loss: 0.0090
Epoch 7/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.9962 - loss: 0.0097 - val_accuracy: 0.9980 - val_loss: 0.0087
Epoch 8/30
432/432 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.9969 - loss: 0.0068 - val_accurac